In [1]:
import argparse
import os, sys
from utils.data_utils import *
from glob import glob
import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm
%matplotlib inline  
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12,9]

from keras.models import Input, Model, Sequential
from keras.layers import Dense
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import LambdaCallback
from sklearn.model_selection import StratifiedKFold, train_test_split

np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [38]:
def is_windows():
    return sys.platform == 'win32'

res_dir = os.path.join('predictions')
if is_windows():
    l_dirs = ['0.21', '0.22', '0.23', '0.24', '0.25']
else:
    l_dirs = ['0.09']
    
def get_data_dir(a, t):
    if t:
        if a:
            data_dir = 'train_ex'
        else:
            data_dir = 'train'
    else:
        if a:
            data_dir = 'valid_ex'
        else:
            data_dir = 'valid'
    return data_dir

def load_loss_df(l_dirs, augmented=False, train=False):
    data_dir = get_data_dir(augmented, train)
    dataframes = []
    for d in l_dirs:
        dataframes.append(pd.read_csv(os.path.join(res_dir, data_dir, d, 'models.csv')))

    df_l = pd.concat(dataframes, ignore_index=True)
    return df_l

def get_gt_name(a, t):
    if t:
        if a:
            name = 'Y_train_augmented.npz'
        else:
            name = 'Y_train.npz'
    else:
        if a:
            name = 'Y_valid_augmented.npz'
        else:
            name = 'Y_valid.npz'
    return name

def load_ground_truth(data_dir, augmented=False, train=False):
    gt_name = get_gt_name(augmented, train)
    y_true_path = os.path.join('augmented', data_dir, gt_name)
    y_true = npz_to_ndarray(np.load(y_true_path))
    return y_true

def load_preds(df_loss, transposed=False):
    preds = []
    for npz_path in df_loss.res_path:
        p = npz_to_ndarray(np.load(npz_path))
        if transposed:
            preds.append(p.transpose())
        else:
            preds.append(p)
    return np.array(preds)

labels = [
    'slash_burn',
    'clear',
    'blooming',
    'primary',
    'cloudy',
    'conventional_mine',
    'water',
    'haze',
    'cultivation',
    'partly_cloudy',
    'artisinal_mine',
    'habitation',
    'bare_ground',
    'blow_down',
    'agriculture',
    'road',
    'selective_logging'
]

In [3]:
valid_augmented = True

In [4]:
df_loss_valid = load_loss_df(l_dirs, augmented=valid_augmented, train=False)
print(len(df_loss_valid))

123


In [5]:
preds_valid = load_preds(df_loss_valid)
print(preds_valid.shape)

(123, 64768, 17)


In [6]:
y_true_valid = load_ground_truth('train_augmented_size_128_mult_8_seed_0', augmented=valid_augmented, train=False)
print(y_true_valid.shape)

(64768, 17)


In [7]:
def get_ensemble_avg_score(ensemble, all_preds, y_true, opt_th=False, opt_step=0.05, th=0.2):
    ensemble_preds = [all_preds[j] for j in ensemble]
    avg_pred = np.mean(ensemble_preds, axis=0)
    if not opt_th:
        avg_pred_final = (avg_pred > th).astype(int)
    else:
        thresholds = find_ratios(y_true, avg_pred, step=opt_step)
        avg_pred_final = (avg_pred > thresholds).astype(int)
    avg_score = fbeta_score(y_true, avg_pred_final, beta=2, average='samples')
    return avg_score

In [8]:
# ensemble = [20]
# ensemble = [20, 62]
# ensemble = [20, 62, 13]
# ensemble = [25, 13, 46]
# ensemble = [20, 25, 13, 46, 51, 8, 12]
# ensemble = [20, 62, 13, 46]
# ensemble = [20, 62, 13, 46, 8]
# ensemble = [20, 62, 13, 46, 8, 22, 12, 51, 25]
ensemble = [110, 58, 108, 13, 89, 112, 119, 86, 22, 66]

In [9]:
preds_ensemble = np.array([preds_valid[j] for j in ensemble])
print("%s" % str(preds_ensemble.shape))
dim_batch_valid = preds_ensemble.shape[2]
dim_classes = preds_ensemble.shape[1]
dim_models = preds_ensemble.shape[0]

(10, 64768, 17)


In [10]:
preds_avg = np.mean(preds_ensemble, axis=0)
preds_final = (preds_avg>0.2).astype(int)
score = fbeta_score(y_true_valid, preds_final, beta=2, average='samples')
print("Score: %.5f" % score)

Score: 0.93029


In [11]:
tp_tn = preds_avg[preds_final == y_true_valid]
tp_tn.shape

(1055804,)

In [12]:
tp = tp_tn[tp_tn > 0.2]
tn = tp_tn[tp_tn <= 0.2]
print("%s %s" % (tp.shape, tn.shape))

(175178,) (880626,)


In [13]:
def show_statistics(data):
    print("Length: %d" % len(data))
    print("Min: %f; Max %f" % (data.min(), data.max()))
    print("Mean: %f; Std %f" % (data.mean(), data.std()))
    print("Mean+2std: %f; Mean-2std %f" % (data.mean() + data.std(), data.mean() - data.std()))

In [14]:
show_statistics(tp)
print("--------------------------------")
show_statistics(tn)

Length: 175178
Min: 0.200046; Max 0.999996
Mean: 0.889833; Std 0.188399
Mean+2std: 1.078232; Mean-2std 0.701434
--------------------------------
Length: 880626
Min: 0.000000; Max 0.199982
Mean: 0.011944; Std 0.029126
Mean+2std: 0.041070; Mean-2std -0.017182


In [15]:
fp_fn = preds_avg[preds_final != y_true_valid]
fp_fn.shape

(45252,)

In [16]:
fp = fp_fn[fp_fn > 0.2]
fn = fp_fn[fp_fn <= 0.2]
print("%s %s" % (fp.shape, fn.shape))

(34262,) (10990,)


In [17]:
show_statistics(fp)
print("--------------------------------")
show_statistics(fn)

Length: 34262
Min: 0.200005; Max 0.998808
Mean: 0.443065; Std 0.207572
Mean+2std: 0.650636; Mean-2std 0.235493
--------------------------------
Length: 10990
Min: 0.000243; Max 0.199994
Mean: 0.091343; Std 0.057310
Mean+2std: 0.148653; Mean-2std 0.034033


In [20]:
correct = preds_avg[np.all(preds_final == y_true_valid, axis=1)]
incorrect = preds_avg[np.any(preds_final != y_true_valid, axis=1)]
print("%s %s" % (correct.shape, incorrect.shape))

(37047, 17) (27721, 17)


In [21]:
range_min = 0.002
range_max = 0.999
correct_in_range = correct[np.all((correct > range_max) | (correct < range_min), axis=1)]
incorrect_in_range = incorrect[np.all((incorrect > range_max) | (incorrect < range_min), axis=1)]
print("%s %s" % (correct_in_range.shape, incorrect_in_range.shape))
print("Misslabeling %.2f%%" % (100*float(incorrect_in_range.shape[0])/(correct_in_range.shape[0] + incorrect_in_range.shape[0])))

(1980, 17) (8, 17)
Misslabeling 0.40%


In [29]:
def improve_score_per_class(preds, cls, fp=True, fn=True):
    p_improved = preds.copy()
    if fp:
        p_improved[(p_improved[:,cls] != y_true_valid[:,cls]) & (p_improved[:,cls] == 1),cls] = 0
    if fn:
        p_improved[(p_improved[:,cls] != y_true_valid[:,cls]) & (p_improved[:,cls] == 0),cls] = 1
    score = fbeta_score(y_true_valid, p_improved, beta=2, average='samples')
    return score

In [52]:
scores_per_class = []
for i in range(17):
    scores_per_class.append(improve_score_per_class(preds_final, i, fp=False, fn=True))
    print("Improved score for class %d: %.5f" % (i, scores_per_class[-1]))

Improved score for class 0: 0.93147
Improved score for class 1: 0.93217
Improved score for class 2: 0.93200
Improved score for class 3: 0.93102
Improved score for class 4: 0.93142
Improved score for class 5: 0.93056
Improved score for class 6: 0.93590
Improved score for class 7: 0.93339
Improved score for class 8: 0.93547
Improved score for class 9: 0.93152
Improved score for class 10: 0.93038
Improved score for class 11: 0.93311
Improved score for class 12: 0.93322
Improved score for class 13: 0.93066
Improved score for class 14: 0.93399
Improved score for class 15: 0.93387
Improved score for class 16: 0.93141


In [53]:
print("------------------------------------------------------")
print("Average improved score %.5f" % np.mean(scores_per_class))
for s in np.sort(scores_per_class):
    i = scores_per_class.index(s)
    print("Improved score for class [%d] - %s: %.5f" % (i, labels[i], s))

------------------------------------------------------
Average improved score 0.93245
Improved score for class [10] - artisinal_mine: 0.93038
Improved score for class [5] - conventional_mine: 0.93056
Improved score for class [13] - blow_down: 0.93066
Improved score for class [3] - primary: 0.93102
Improved score for class [16] - selective_logging: 0.93141
Improved score for class [4] - cloudy: 0.93142
Improved score for class [0] - slash_burn: 0.93147
Improved score for class [9] - partly_cloudy: 0.93152
Improved score for class [2] - blooming: 0.93200
Improved score for class [1] - clear: 0.93217
Improved score for class [11] - habitation: 0.93311
Improved score for class [12] - bare_ground: 0.93322
Improved score for class [7] - haze: 0.93339
Improved score for class [15] - road: 0.93387
Improved score for class [14] - agriculture: 0.93399
Improved score for class [8] - cultivation: 0.93547
Improved score for class [6] - water: 0.93590
